# 20. 나라면 이런 거 사겠다 (1) - NCAV

[20강의](https://www.youtube.com/watch?v=6B2U2uJRJ0A) 를 파이썬으로 구현해보겠습니다.

## 설치

Pandas 가 있어야 합니다. 설치가 안되어있으시면 다음 셀을 실행시키세요.

In [1]:
import sys
!{sys.executable} -m pip install pandas xlrd openpyxl 2>&1 >/dev/null

## 다운로드

[네이버 카페 퀀트킹(QuantKing)](https://cafe.naver.com/quantking)에서 최신 엑셀 파일을 다운받으세요.

저는 **QuantKing(2019.02.15).xlsb** 파일을 받았습니다. 엑셀로 여셔서 **QuantKing.xlsx** 파일로 다시 저장해주세요.

## 파일 로딩

파일을 열어서 조사해 보겠습니다. Pandas의 read_excel() 이란 함수를 이용하면 열 수 있네요.

In [2]:
import pandas as pd

In [3]:
filepath = 'QuantKing.xlsx'
df = pd.read_excel(filepath, sheet_name='퀀트데이타', skiprows=2).drop('Unnamed: 0', axis=1)
df.head()

,코드 번호,회사명,업종 (대),업종 (소),코스피코스닥,주가 (원),시가총액 (억),상장주식수 (만주),자사주 (만주),자사주 비중 (%),...,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,19년 1Q(E).2,18년4Q 매출액,18년4Q 영업이익,18년4Q 순이익,18년4Q 매출액.1,18년4Q 영업이익.1,18년4Q 순이익.1
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,46050,2749085,596978.0,0,0.0,...,109815,129674,83301.00,67567.0,592650.00,108006.00,83301.00,630816.0,132917.0,100074.0
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,73800,537266,72800.0,4400,6.0,...,43299,46936,33962.90,15272.0,99380.81,44300.75,33962.90,100565.0,50087.0,41127.0
2,A068270,셀트리온,제약 및 바이오,바이오,코스피,209500,268052,12795.0,85,0.7,...,840,562,530.00,633.0,NaN,NaN,NaN,2514.0,785.0,530.0
3,A051910,LG화학,화학,종합 화학,코스피,380000,268251,7059.0,36,0.5,...,4777,3434,1201.46,3250.0,73427.12,2895.59,1201.46,67891.0,3136.0,1906.0
4,A005380,현대차,자동차 및 관련부품,완성차,코스피,121000,258539,21367.0,939,4.4,...,7006,2692,-1297.74,7997.0,256695.34,5011.27,-1297.74,260649.0,7517.0,6726.0


강의에서 사용하는 컬럼만 잘라서 보관하도록 하겠습니다.

<span style="color:red">**추후수정: "18년 1Q" 라는 비슷한 컬럼이 여럿 있습니다. ".2"가 붙은게 순이익이고, 나머지는 영업이익, 매출관련입니다. 잘선택해서 쓰세요.**</span>

In [4]:
df = df[['회사명',
         '본사\n국내\n=1', # 중국여부
         '청산가치비율\n(NCAV전략)\n(%)', # 청산가치/시가총액 비율h
         '단순\n부채비율\n(%)', # 부채비율
         '15년\n순이익', '16년\n순이익', '17년\n순이익', '18년 1Q.2', '18년 2Q.2', '18년 3Q.2', '18년 4Q(E).2', # 흑자기업
         '고점\nPBR', # 과거영광
         'F스코어\n신주발행X\n여부', # 주식발행여부 (equity financing)
         '유동\n비율\n(%)', # 유동비율
         '시가총액\n(억)', # 시가총액
         '시가\n배당율 고점' # 2번전략
        ]].set_index('회사명')
df_original = df
df_original.head()

,본사 국내 =1,청산가치비율 (NCAV전략) (%),단순 부채비율 (%),15년 순이익,16년 순이익,17년 순이익,18년 1Q.2,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,고점 PBR,F스코어 신주발행X 여부,유동 비율 (%),시가총액 (억),시가 배당율 고점
회사명,,,,,,,,,,,,,,,
삼성전자,1,29.5,39,186946,224157,413446,116118,109815,129674,83301.00,2.07,0,235.968130,2749085,2.36
SK하이닉스,1,7.1,32,43224,29538,106415,31203,43299,46936,33962.90,2.89,1,176.969777,537266,2.24
셀트리온,1,3.1,31,1541,1780,3994,834,840,562,530.00,15.48,0,258.906134,268052,0.74
LG화학,1,8.5,65,11530,12811,19453,5313,4777,3434,1201.46,3.80,1,183.438248,268251,3.47
현대차,1,-220.3,140,64173,54064,40328,6680,7006,2692,-1297.74,1.55,1,101.592719,258539,2.91


## NCAV 전략1: Graham/Bleker

전통적인 **우량** NCAV 매수전략
1. 중국은 안된다 (미국애들도 중국 회계는 쓰레기인지 안다!)
2. 청산가치 / 시가총액 비율이 높으면 굿! (벤자민 그레이엄은 150% 넘으면 좋다고)
3. 낮은 부채비율 (100% 이하라고 합시다)
4. 과거 흑자기업
5. 가격이 NCAV 보다 높았던 역사 (PBR 0.8정도)
6. 주식발행 없음
7. 유동비율 굿 (150% 이상)
8. 소형주

출처: [16 Benjamin Graham Inspired Rules For Net Net Investing](https://www.valuewalk.com/2014/04/16-benjamin-graham-inspired-rules-for-net-net-investing/)

우선 청산가치가 높은 기업부터 보시죠. 벤자민 그레이엄은 150%, 강환국 선생님은 100% 넘는 기업을 전부 본다고 하십니다.

In [5]:
df = df.sort_values(by='청산가치비율\n(NCAV전략)\n(%)', ascending=False)
df.head(n=10)

,본사 국내 =1,청산가치비율 (NCAV전략) (%),단순 부채비율 (%),15년 순이익,16년 순이익,17년 순이익,18년 1Q.2,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,고점 PBR,F스코어 신주발행X 여부,유동 비율 (%),시가총액 (억),시가 배당율 고점
회사명,,,,,,,,,,,,,,,
차이나그레이트,0,1153.6,13,749,549,-162,155,140,53,NaN,1.98,1,786.709515,386,0.00
씨케이에이치,0,1037.2,7,735,524,8,-192,-119,-177,NaN,1.78,1,1153.391763,358,0.00
이스트아시아홀딩스,0,641.5,8,35,-96,28,8,13,12,NaN,1.28,0,1645.265625,258,0.00
KJ프리텍,1,443.8,70,47,-31,-86,-32,-33,-60,NaN,3.16,0,180.656328,96,0.00
감마누,1,425.0,151,13,-33,-27,3,13,-113,NaN,5.69,1,218.927200,100,0.00
에스앤씨엔진그룹,0,356.2,5,295,279,200,37,59,43,NaN,1.71,0,1196.223610,623,0.00
세원정공,1,333.1,25,427,395,27,-26,56,-61,NaN,1.21,1,348.181748,756,1.67
KISCO홀딩스,1,320.7,19,437,459,129,70,119,-41,NaN,0.61,0,483.409575,2522,2.72
SJM홀딩스,1,265.4,18,112,57,153,7,18,11,NaN,0.75,1,502.442978,503,6.32


맨위를 보면 차이나그레이트네요. 중국기업이니 당연히 빼줘야 합니다.

다른 필터들도 걸어줍니다.
- 부채비율 100% 이하
- 과거 흑자기업 (15, 16, 17년 순수익 > 0, 18년합 > 0)
- 과거 NCAV 보다 높았던 역사 (고점 PBR > 0.8)
- 신주발행 없음 (F 스코어 신주발행 여부 = 1)
- 유동비율 > 150%
- 소형주

In [6]:
df = df[df['본사\n국내\n=1'] == 1]
df = df[df['청산가치비율\n(NCAV전략)\n(%)'] > 100]
df = df[df['단순\n부채비율\n(%)'] < 100]
df = df[df['15년\n순이익'] > 0]
df = df[df['16년\n순이익'] > 0]
df = df[df['17년\n순이익'] > 0]
df['18년 합'] = df['18년 1Q.2'] + df['18년 2Q.2'] + df['18년 3Q.2'] + df['18년 4Q(E).2'].fillna(0)
df = df[df['18년 합'] > 0]
df = df[df['고점\nPBR'] > 0.8]
df = df[df['F스코어\n신주발행X\n여부'] == 1]
df = df[df['유동\n비율\n(%)'] > 150]
df = df.sort_values(by='청산가치비율\n(NCAV전략)\n(%)', ascending=False).iloc[:200]
df.head(n=10)

,본사 국내 =1,청산가치비율 (NCAV전략) (%),단순 부채비율 (%),15년 순이익,16년 순이익,17년 순이익,18년 1Q.2,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,고점 PBR,F스코어 신주발행X 여부,유동 비율 (%),시가총액 (억),시가 배당율 고점,18년 합
회사명,,,,,,,,,,,,,,,,
아이디스홀딩스,1,229.6,34,155,17,117,39,56,57,NaN,3.69,1,361.423333,1438,2.31,152.0
CS홀딩스,1,208.4,12,216,92,59,17,15,14,NaN,0.97,1,791.790688,753,1.27,46.0
SJM,1,190.0,19,219,150,57,-2,32,6,NaN,1.44,1,508.423681,520,4.54,36.0
동일산업,1,172.4,15,42,134,233,77,52,33,NaN,0.81,1,632.867783,1428,3.11,162.0
동원개발,1,164.6,19,902,980,1006,172,278,331,253.0,1.37,1,680.500000,3864,5.73,1034.0
동양이엔피,1,145.2,36,177,155,52,4,39,18,NaN,1.06,1,329.983749,798,6.88,61.0
삼양통상,1,144.1,13,105,229,249,50,55,62,NaN,1.66,1,876.463479,1326,3.99,167.0
예림당,1,136.3,52,23,54,196,217,-36,9,NaN,3.92,1,257.919690,1516,8.36,190.0
신도리코,1,135.7,11,94,318,317,64,98,95,NaN,0.93,1,1013.106739,5010,5.06,257.0


강의와 리스트가 비슷합니다.

홀딩스는 회계 방법이 다르기에 청산가치 비율이 높게 나올 수 있습니다. 즉, <span style="color:red">**홀딩스 회사들은 NCAV 전략에서 제외해주셔야 합니다.**</span> (그런데, 2018년 4분기를 합하니 홀딩스가 아예 없네요.)

여기에 소형주를 제외하셔도 됩니다. 동원개발은 거의 4000억인 회사니 제외해도 되겠네요.

퀀트킹 종목현황 탭에 가셔서 회사 하나하나를 보세요. [강의에서 선생님이 고르시는걸 보면](https://www.youtube.com/watch?v=6B2U2uJRJ0A&t=840s) 이해가 더 잘됩니다.

## NCAV 전략2: Oppenheimer/Carlisle

**비유량** NCAV 주식 수입이 높아
1. 적자
2. 무배당
3. 청산가치 /시가총액 비율이 높으면 굿! (상위 20% 제외)

- 우선, 청산가치 비율 150으로 자르는건 동일하구요.
- 상위 6개 중 4개가 짱깨니 6개 자르구요 (상위 20% 제외)
- 4년연속 적자는 너무하니, 최근 2년 적자로 자르구요.
- 배당도 0인 기업으로 자릅니다

In [7]:
df = df_original
df = df[df['청산가치비율\n(NCAV전략)\n(%)'] > 150]
df = df.sort_values(by='청산가치비율\n(NCAV전략)\n(%)', ascending=False).iloc[6:]
df = df[df['17년\n순이익'] < 0]
df['18년 합'] = df['18년 1Q.2'] + df['18년 2Q.2'] + df['18년 3Q.2'] #+ df['18년 4Q(E)'].fillna(0)
df = df[df['18년 합'] < 0]
df = df[df['시가\n배당율 고점'] == 0]
df

,본사 국내 =1,청산가치비율 (NCAV전략) (%),단순 부채비율 (%),15년 순이익,16년 순이익,17년 순이익,18년 1Q.2,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,고점 PBR,F스코어 신주발행X 여부,유동 비율 (%),시가총액 (억),시가 배당율 고점,18년 합
회사명,,,,,,,,,,,,,,,,
에프티이앤이,1,206.8,68,-223,-10,-85,-33,-24,-23,NaN,9.7,0,139.557393,59,0.0,-80


~~제 테스트로는 아무 회사도 안나오네요. 한국에 아주 쓰레기는 없나봅니다.~~

~~강의에서는 에프티이앤이가 나왔었는데요. 강의에서 마지막에 보이는 에프티이엔이와 아래 자료가 다르네요. 특히나 18년 쿼터별 값이 다릅니다. 아마 백테스트중에 또 실수하신게 아닌가 싶습니다.~~

추후추가: 제 테스트가 틀렸네요. 강의처럼 에프티이엔이가 나옵니다. (어차피 2018년 10월쯤 상폐되었네요.)

In [8]:
df_original.loc['에프티이앤이']

본사\n국내\n=1                 1.000000
청산가치비율\n(NCAV전략)\n(%)    206.800000
단순\n부채비율\n(%)             68.000000
15년\n순이익                -223.000000
16년\n순이익                 -10.000000
17년\n순이익                 -85.000000
18년 1Q.2                 -33.000000
18년 2Q.2                 -24.000000
18년 3Q.2                 -23.000000
18년 4Q(E).2                     NaN
고점\nPBR                    9.700000
F스코어\n신주발행X\n여부            0.000000
유동\n비율\n(%)              139.557393
시가총액\n(억)                 59.000000
시가\n배당율 고점                 0.000000
Name: 에프티이앤이, dtype: float64